# Section 1: Finesse3 Interface

In this project, we search for the optical configuration of a gravitational wave interferometer optimised for the detection of gravitational waves emitted by the post-merger remnants of binary neutron star (BNS) mergers. When the post-merger remnant is a neutron star, its extreme temperature and density provide an unparalleled cosmic laboratory to explore the (hot) nuclear equation of state. We numerically model many possible detector designs by varying optical parameters and inject numerical relativity waveforms of BNS mergers to calculate the expected detection rate of post-merger signals. We find that the configuration maximising this detection rate improves on the calculated detection rate for NEMO. These results are only preliminary and there is much more work to be done (see all the TODOs scattered throughout these notebooks). Nonetheless, I hope that the reader finds the notebook a stepping stone towards bringing NEMO to life. 

A lot of the learning process involved in understanding gravitational wave interferometery was spread throughout my daily notebooks and is not recorded directly in these results. I am more than happy to take questions via e-mail at juyu0917@uni.sydney.edu.au.

In [2]:
import finesse
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import matplotlib.pyplot as plt
from IPython.display import display
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from scipy.signal import find_peaks, peak_widths
from scipy import interpolate

save_path = "./SensitivityCurves/"
default_srmT = 0.04827586206896554
default_srmPhi = -90.02274126019697
default_srcL = 354
default_itmT = 0.01397

store_prmT = 0
store_lasPow = 0
store_itmT = 0

def plot(srmT, srmPhi, srcL, itmT): # This executes when 'Interact' is clicked
    global store_prmT
    global store_lasPow
    global store_itmT

    # Initialise variables
    # if no peak is found a log error will be thrown for the maximum and half-maximum lines but the sensitivity curves are OK)
    fsig = np.geomspace(100,1e4, 201)
    peak_sens = 0
    peak_f = 0
    peak_bw = 0
    left_f = 0
    right_f = 0

    srmT = float(srmT)
    srmPhi = float(srmPhi)
    srcL = float(srcL)
    itmT = float(itmT)

    # If itmT changes, then optimise prmT and lasPow
    if itmT != store_itmT:
        peakPow, prmT, lawPow = find_optimal_prmT(srmT, srmPhi, srcL, itmT)
        store_prmT = prmT
        store_lasPow = lasPow
        store_itm = itmT
    else:
        prmT = store_prmT
        lasPow = store_lasPow

    kat = finesse.Model()
    # From SRC-tunability_DCreadout_changeBW_fin3exp.ipynb
    kat.parse(
    f"""
    # NEMO Base Model (simplified from OzHF_ITMloss_4km.kat and converted to Finesse 3
    ###########################################################################
    ###   Variables
    ###########################################################################
    var Larm 4000
    var Mtm 74.1 # From NEMO paper (94.4 in OzHF_ITMloss4km.kat)
    var itmT {itmT}
    var lmichx 4.5
    var lmichy 4.45

    ###########################################################################
    ###   Input optics
    ###########################################################################
    l L0 {lasPow}

    s l_in L0.p1 prm.p1
    # Power recycling mirror
    m prm T={prmT} L=2e-5 phi=90
    s prc prm.p2 bs.p1 L=53

    # Central beamsplitter
    bs bs R=0.4999625 T=0.4999625 alpha=45

    # CHECK Input laser power
    pd P_in L0.p1.o
    # CHECK Laser power incident on BS
    pd P_BS bs.p1.i
    # CHECK PRC Power
    pd P_PRC bs.p1.o

    ###########################################################################
    ###   X arm
    ###########################################################################
    s lx bs.p3 itmxar.p1 L=lmichx

    m itmxar T=1-265.0e-6 L=265.0e-6 phi=180
    s ar_thick itmxar.p2 itmx.p1 L=0
    m itmx T=itmT L=20u phi=180
    s LX itmx.p2 etmx.p1 L=Larm

    m etmx T=5u L=20u phi=179.99999

    pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
    pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

    # CHECK X-arm cavity power
    pd P_armX etmx.p1.i

    ###########################################################################
    ###   Y arm
    ###########################################################################
    s ly bs.p2 itmyar.p1 L=lmichy

    m itmyar T=1-265.0e-06 L=265.0e-06 phi=90 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
    s ar_thicky itmyar.p2 itmy.p1 L=0
    m itmy T=itmT L=20u phi=90
    s LY itmy.p2 etmy.p1 L=Larm

    m etmy T=5u L=20u phi=90.00001

    pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
    pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

    # CHECK Y-arm cavity power
    pd P_armY etmy.p1.i

    ###########################################################################
    ###   SRM
    ###########################################################################
    s src bs.p4 srm.p1 L={srcL}
    m srm R={0.99985-srmT} T={srmT} phi={srmPhi}

    # CHECK SRC power
    pd P_SRC srm.p1.i

    ###########################################################################
    ###   Output & squeezing (from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb)
    ###########################################################################
    dbs OFI 
    link(srm.p2, OFI.p1)
    readout_dc AS OFI.p3.o

    # A squeezed source could be injected into the dark port
    sq sqz db=7 angle=90
    link(sqz, OFI.p2)

    # ------------------------------------------------------------------------------
    # Degrees of Freedom
    # ------------------------------------------------------------------------------
    dof STRAIN LX.dofs.h +1  LY.dofs.h -1

    # signal generator
    sgen sig STRAIN

    qnoised NSR_with_RP AS.p1.i nsr=True
    qshot NSR_without_RP AS.p1.i nsr=True
    pd1 signal AS.p1.i f=fsig

    fsig(1)
    xaxis(fsig, log, 100, 10k, 200)
    """
    )
    out = kat.run()
    neg_sens = -np.abs(out['NSR_with_RP']) # Take reciprocal sensitivity to use findpeaks i.e. FWHM defined by using reciprocal sensitivity!
    peak_idxs, _ = find_peaks(neg_sens)
    
    if peak_idxs.size != 0: # If a peak is found (first peak taken)
        fwhm_idxs = peak_widths(neg_sens, peak_idxs, rel_height=0.5)
        left_idx = fwhm_idxs[2][0]
        right_idx = fwhm_idxs[3][0]
        interp_fsig = interpolate.interp1d(np.arange(201), fsig)
        
        left_f = interp_fsig(left_idx)
        right_f = interp_fsig(right_idx)
        peak_sens = np.abs(out['NSR_with_RP'])[peak_idxs[0]]
        peak_f = fsig[peak_idxs[0]]
        peak_bw = right_f - left_f
    
    fig_qnoise = go.Figure()
    fig_qnoise.add_trace(go.Scatter(x=fsig, y=np.abs(out['NSR_with_RP']),mode='lines+markers',name='qnoised NSR'))
    fig_qnoise.add_trace(go.Scatter(x=fsig, y=np.abs(out['NSR_without_RP']),mode='lines+markers',name='qshot NSR'))
    fig_qnoise.update_xaxes(type="log")
    fig_qnoise.update_yaxes(type="log")
    fig_qnoise.add_vline(x=peak_f)
    fig_qnoise.add_vline(x=right_f,line_dash='dash',line_color='green')
    fig_qnoise.add_vline(x=left_f,line_dash='dash',line_color='green')
    fig_qnoise.update_layout(title="ASD (qnoised, qshot)",xaxis_title="Frequency [Hz]",yaxis_title="Sensitivity [1/rt Hz]")
    fig_qnoise.show()
    
    fig_signal = go.Figure()
    fig_signal.add_trace(go.Scatter(x=fsig, y=np.abs(out['signal']),mode='lines+markers'))
    fig_signal.update_xaxes(type="log")
    fig_signal.update_yaxes(type="log")
    fig_signal.add_vline(x=peak_f)
    fig_signal.add_vline(x=right_f,line_dash='dash',line_color='green')
    fig_signal.add_vline(x=left_f,line_dash='dash',line_color='green')
    fig_signal.update_layout(title="Signal Gain (pd1)",xaxis_title="Frequency [Hz]",yaxis_title="Power [W]")
    fig_signal.show()
    
    # Print model outputs
    print(f"Optimal prmT: {prmT}")
    print(f"Peak Sensitivity: {peak_sens} 1/rt Hz, Peak Frequency: {peak_f}Hz, Peak FWHM: {peak_bw}Hz")
    print(f"Input laser power: {np.max(np.abs(out['P_in']))}W")
    print(f"PRC power: {np.max(np.abs(out['P_PRC']))*1e-3}kW")
    print(f"Laser power incident on BS: {np.max(np.abs(out['P_BS']))*1e-3}kW")
    print(f"X-arm cavity power: {np.max(np.abs(out['P_armX']))*1e-6}MW")
    print(f"Y-arm cavity power: {np.max(np.abs(out['P_armY']))*1e-6}MW")
    print(f"SRC power: {np.max(np.abs(out['P_SRC']))}W")

    def save_curve(arg):
        lines = [f"{fsig[i]} {np.abs(out['NSR_with_RP'])[i]}\n" for i in range(201)]
        lines[-1].rstrip('\n')
        filename = f"SRM_{srmT}_{srmPhi}_{srcL}_{itmT}_{prmT}_{lasPow}_ASD_with_RP.txt"
        file = open(save_path+filename, "w")
        file.writelines(lines)
        file.close()
        print(f"Saved {filename}!")

    button_save = widgets.Button(description = 'Save')   
    button_save.on_click(save_curve)
    display(button_save)

# Default values here
widgets.interact_manual(plot, srmT=widgets.Text(value=f"{default_srmT}"),
                 srmPhi=widgets.Text(value=f"{default_srmPhi}"), srcL=widgets.Text(value=f"{default_srcL}"), itmT=widgets.Text(value=f"{default_itmT}"));

# Auto-tune prmT with findpeaks to maximise arm cavity power (impedance matching)
def find_optimal_prmT(srmT,srmPhi,srcL,itmT):
    armPow = 4.5e6
    lasPow = 500
    
    vary_prmT = np.geomspace(0.01,0.5,100)
    circX = np.zeros((100,))
    # Find prmT to maximise arm cavity power
    for i, prmT in enumerate(vary_prmT):
        kat = finesse.Model()
        kat.parse(
        f"""
        # NEMO Base Model (simplified from OzHF_ITMloss_4km.kat and converted to Finesse 3)
        ###########################################################################
        ###   Variables
        ###########################################################################
        var Larm 4000
        var Mtm  74.1 # from NEMO paper (94.4 in OzHF_ITMloss_4km.kat)
        var itmT {itmT}
        var lmichx 4.5
        var lmichy 4.45

        ###########################################################################
        ###   Input optics
        ###########################################################################
        l L0 500

        s l_in L0.p1 prm.p1
        # Power recycling mirror
        m prm T={prmT} L=2e-05 phi=90
        s prc prm.p2 bs.p1 L=53

        # Central beamsplitter
        bs bs R=0.4999625 T=0.4999625 alpha=45

        # CHECK Input laser power
        # pd P_in L0.p1.o
        # CHECK Laser power incident on BS
        # pd P_BS bs.p1.i
        # CHECK PRC Power
        # pd P_PRC bs.p1.o

        ###########################################################################
        ###   X arm
        ###########################################################################
        s lx bs.p3 itmxar.p1 L=lmichx

        m itmxar T=1-265.0e-06 L=265.0e-06 phi=180 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
        s ar_thick itmxar.p2 itmx.p1 L=0
        m itmx T=itmT L=20u phi=180
        s LX itmx.p2 etmx.p1 L=Larm

        m etmx T=5u L=20u phi=179.99999

        pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
        pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

        # CHECK X-arm cavity power
        pd P_armX etmx.p1.i

        ###########################################################################
        ###   Y arm
        ###########################################################################
        s ly bs.p2 itmyar.p1 L=lmichy

        m itmyar T=1-265.0e-06 L=265.0e-06 phi=90 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
        s ar_thicky itmyar.p2 itmy.p1 L=0
        m itmy T=itmT L=20u phi=90
        s LY itmy.p2 etmy.p1 L=Larm

        m etmy T=5u L=20u phi=90.00001

        pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
        pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

        # CHECK Y-arm cavity power
        # pd P_armY etmy.p1.i

        ###########################################################################
        ###   SRM
        ###########################################################################
        s src bs.p4 srm.p1 L={srcL}
        m srm R={0.99985-srmT} T={srmT} phi={srmPhi}

        # CHECK SRC power
        # pd P_SRC srm.p1.i

        ###########################################################################
        ###   Output & squeezing (from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb)
        ###########################################################################
        noxaxis()
        """
        )
        out = kat.run()
        circX[i] = out['P_armX']
    
    peak_power = circX[np.argmax(circX)]
    peak_T = vary_prmT[np.argmax(circX)]
    
    if peak_power < armPow:
        while peak_power < armPow and lasPow < 500*1.10:
            lasPow += 1
            kat = finesse.Model()
            kat.parse(
            f"""
            # NEMO Base Model (simplified from OzHF_ITMloss_4km.kat and converted to Finesse 3)
            ###########################################################################
            ###   Variables
            ###########################################################################
            var Larm 4000
            var Mtm  74.1 # from NEMO paper (94.4 in OzHF_ITMloss_4km.kat)
            var itmT {itmT}
            var lmichx 4.5
            var lmichy 4.45

            ###########################################################################
            ###   Input optics
            ###########################################################################
            l L0 {lasPow}

            s l_in L0.p1 prm.p1
            # Power recycling mirror
            m prm T={peak_T} L=2e-05 phi=90
            s prc prm.p2 bs.p1 L=53

            # Central beamsplitter
            bs bs R=0.4999625 T=0.4999625 alpha=45

            # CHECK Input laser power
            # pd P_in L0.p1.o
            # CHECK Laser power incident on BS
            # pd P_BS bs.p1.i
            # CHECK PRC Power
            # pd P_PRC bs.p1.o

            ###########################################################################
            ###   X arm
            ###########################################################################
            s lx bs.p3 itmxar.p1 L=lmichx

            m itmxar T=1-265.0e-06 L=265.0e-06 phi=180 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
            s ar_thick itmxar.p2 itmx.p1 L=0
            m itmx T=itmT L=20u phi=180
            s LX itmx.p2 etmx.p1 L=Larm

            m etmx T=5u L=20u phi=179.99999

            pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
            pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

            # CHECK X-arm cavity power
            pd P_armX etmx.p1.i

            ###########################################################################
            ###   Y arm
            ###########################################################################
            s ly bs.p2 itmyar.p1 L=lmichy

            m itmyar T=1-265.0e-06 L=265.0e-06 phi=90 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
            s ar_thicky itmyar.p2 itmy.p1 L=0
            m itmy T=itmT L=20u phi=90
            s LY itmy.p2 etmy.p1 L=Larm

            m etmy T=5u L=20u phi=90.00001

            pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
            pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

            # CHECK Y-arm cavity power
            # pd P_armY etmy.p1.i

            ###########################################################################
            ###   SRM
            ###########################################################################
            s src bs.p4 srm.p1 L={srcL}
            m srm R={0.99985-srmT} T={srmT} phi={srmPhi}

            # CHECK SRC power
            # pd P_SRC srm.p1.i

            ###########################################################################
            ###   Output & squeezing (from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb)
            ###########################################################################
            noxaxis()
            """
            )
            out = kat.run()
            peak_power = out['P_armX']
    else: 
        while peak_power > armPow:
            lasPow -= 1
            kat = finesse.Model()
            kat.parse(
            f"""
            # NEMO Base Model (simplified from OzHF_ITMloss_4km.kat and converted to Finesse 3)
            ###########################################################################
            ###   Variables
            ###########################################################################
            var Larm 4000
            var Mtm  74.1 # from NEMO paper (94.4 in OzHF_ITMloss_4km.kat)
            var itmT {itmT}
            var lmichx 4.5
            var lmichy 4.45

            ###########################################################################
            ###   Input optics
            ###########################################################################
            l L0 {lasPow}

            s l_in L0.p1 prm.p1
            # Power recycling mirror
            m prm T={peak_T} L=2e-05 phi=90
            s prc prm.p2 bs.p1 L=53

            # Central beamsplitter
            bs bs R=0.4999625 T=0.4999625 alpha=45

            # CHECK Input laser power
            # pd P_in L0.p1.o
            # CHECK Laser power incident on BS
            # pd P_BS bs.p1.i
            # CHECK PRC Power
            # pd P_PRC bs.p1.o

            ###########################################################################
            ###   X arm
            ###########################################################################
            s lx bs.p3 itmxar.p1 L=lmichx

            m itmxar T=1-265.0e-06 L=265.0e-06 phi=180 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
            s ar_thick itmxar.p2 itmx.p1 L=0
            m itmx T=itmT L=20u phi=180
            s LX itmx.p2 etmx.p1 L=Larm

            m etmx T=5u L=20u phi=179.99999

            pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
            pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

            # CHECK X-arm cavity power
            pd P_armX etmx.p1.i

            ###########################################################################
            ###   Y arm
            ###########################################################################
            s ly bs.p2 itmyar.p1 L=lmichy

            m itmyar T=1-265.0e-06 L=265.0e-06 phi=90 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
            s ar_thicky itmyar.p2 itmy.p1 L=0
            m itmy T=itmT L=20u phi=90
            s LY itmy.p2 etmy.p1 L=Larm

            m etmy T=5u L=20u phi=90.00001

            pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
            pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

            # CHECK Y-arm cavity power
            # pd P_armY etmy.p1.i

            ###########################################################################
            ###   SRM
            ###########################################################################
            s src bs.p4 srm.p1 L={srcL}
            m srm R={0.99985-srmT} T={srmT} phi={srmPhi}

            # CHECK SRC power
            # pd P_SRC srm.p1.i

            ###########################################################################
            ###   Output & squeezing (from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb)
            ###########################################################################
            noxaxis()
            """
            )
            out = kat.run()
            peak_power = out['P_armX']
        lasPow += 1
    
    return peak_power, peak_T, lasPow

interactive(children=(Text(value='0.04827586206896554', continuous_update=False, description='srmT'), Text(val…

## 1.2 vSRM Configuration Dashboard

In [3]:
import finesse
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import matplotlib.pyplot as plt
from IPython.display import display
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from scipy.signal import find_peaks, peak_widths
from scipy import interpolate

save_path = "./SensitivityCurves/"
default_phiComm = 0
default_phiDiff = 6.3565
default_srcL = 354
default_itmT = 0.01397

store_prmT = 0
store_lasPow = 0
store_itmT = 0

def plot(phiComm,phiDiff,srcL,itmT): # This executes when 'Interact' is clicked    
    global store_prmT
    global store_lasPow
    global store_itmT
    
    # Initialise variables (
    # if no peak is found a log error will be thrown for the maximum and half-maximum lines but the sensitivity curves are OK)
    fsig = np.geomspace(100,10e3,201)
    peak_sens = 0
    peak_f = 0
    peak_bw = 0
    left_f = 0
    right_f = 0
    
    phiComm = float(phiComm)
    phiDiff = float(phiDiff)
    srcL = float(srcL)
    itmT = float(itmT)
    
    # If itmT changes, then optimise prmT and lasPow
    if itmT != store_itmT:
        peakPow, prmT, lasPow = find_optimal_prmT(phiComm,phiDiff,srcL,itmT)
        store_prmT = prmT
        store_lasPow = lasPow
        store_itmT = itmT
    else:
        prmT = store_prmT
        lasPow = store_lasPow
    
    kat = finesse.Model()
    # From SRC_tunability_DCreadout_changedBW_fin3exp.ipynb
    kat.parse(
    f"""
    # NEMO Base Model (simplified from OzHF_ITMloss_4km.kat and converted to Finesse 3)
    ###########################################################################
    ###   Variables
    ###########################################################################
    var Larm 4000
    var Mtm  74.1 # from NEMO paper (94.4 in OzHF_ITMloss_4km.kat)
    var itmT {itmT}
    var lmichx 4.5
    var lmichy 4.45

    ###########################################################################
    ###   Input optics
    ###########################################################################
    l L0 {lasPow}

    s l_in L0.p1 prm.p1
    # Power recycling mirror
    m prm T={prmT} L=2e-05 phi=90
    s prc prm.p2 bs.p1 L=53

    # Central beamsplitter
    bs bs R=0.4999625 T=0.4999625 alpha=45

    # CHECK Input laser power
    pd P_in L0.p1.o
    # CHECK Laser power incident on BS
    pd P_BS bs.p1.i
    # CHECK PRC Power
    pd P_PRC bs.p1.o

    ###########################################################################
    ###   X arm
    ###########################################################################
    s lx bs.p3 itmxar.p1 L=lmichx

    m itmxar T=1-265.0e-06 L=265.0e-06 phi=180 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
    s ar_thick itmxar.p2 itmx.p1 L=0
    m itmx T=itmT L=20u phi=180
    s LX itmx.p2 etmx.p1 L=Larm

    m etmx T=5u L=20u phi=179.99999

    pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
    pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

    # CHECK X-arm cavity power
    pd P_armX etmx.p1.i

    ###########################################################################
    ###   Y arm
    ###########################################################################
    s ly bs.p2 itmyar.p1 L=lmichy

    m itmyar T=1-265.0e-06 L=265.0e-06 phi=90 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
    s ar_thicky itmyar.p2 itmy.p1 L=0
    m itmy T=itmT L=20u phi=90
    s LY itmy.p2 etmy.p1 L=Larm

    m etmy T=5u L=20u phi=90.00001

    pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
    pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

    # CHECK Y-arm cavity power
    pd P_armY etmy.p1.i

    ###########################################################################
    ###   vSRM
    ###########################################################################
    s src bs.p4 SRC_BS.p1 L={srcL}
    bs SRC_BS T=0.5 L=0 alpha=45
    s vSRC1 SRC_BS.p2 vSRM1.p1 L=4.5
    m vSRM1 T=0 L=0 phi={-90+phiComm+phiDiff}
    s vSRC2 SRC_BS.p3 vSRM2.p1 L=4.5
    m vSRM2 T=0 L=0 phi={0+phiComm-phiDiff}

    # CHECK SRC power
    pd P_SRC SRC_BS.p1.i

    ###########################################################################
    ###   Output & squeezing (from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb)
    ###########################################################################
    dbs OFI 
    link(SRC_BS.p4, OFI.p1)
    readout_dc AS OFI.p3.o

    # A squeezed source could be injected into the dark port
    sq sqz db=-7 angle=90
    link(sqz, OFI.p2)

    # ------------------------------------------------------------------------------
    # Degrees of Freedom
    # ------------------------------------------------------------------------------
    dof STRAIN LX.dofs.h +1  LY.dofs.h -1

    # signal generator
    sgen sig STRAIN

    qnoised NSR_with_RP AS.p1.i nsr=True
    qshot NSR_without_RP AS.p1.i nsr=True
    pd1 signal AS.p1.i f=fsig

    fsig(1)
    xaxis(fsig, log, 100, 10k, 200)
    """
    )
    out = kat.run()
    neg_sens = -np.abs(out['NSR_with_RP']) # Take reciprocal sensitivity to use findpeaks i.e. FWHM defined by using reciprocal sensitivity!
    peak_idxs, _ = find_peaks(neg_sens)
    
    if peak_idxs.size != 0: # If a peak is found (first peak taken)
        fwhm_idxs = peak_widths(neg_sens, peak_idxs, rel_height=0.5)
        left_idx = fwhm_idxs[2][0]
        right_idx = fwhm_idxs[3][0]
        interp_fsig = interpolate.interp1d(np.arange(201), fsig)
        
        left_f = interp_fsig(left_idx)
        right_f = interp_fsig(right_idx)
        peak_sens = np.abs(out['NSR_with_RP'])[peak_idxs[0]]
        peak_f = fsig[peak_idxs[0]]
        peak_bw = right_f - left_f
    
    fig_qnoise = go.Figure()
    fig_qnoise.add_trace(go.Scatter(x=fsig, y=np.abs(out['NSR_with_RP']),mode='lines+markers',name='qnoised NSR'))
    fig_qnoise.add_trace(go.Scatter(x=fsig, y=np.abs(out['NSR_without_RP']),mode='lines+markers',name='qshot NSR'))
    fig_qnoise.update_xaxes(type="log")
    fig_qnoise.update_yaxes(type="log")
    fig_qnoise.add_vline(x=peak_f)
    fig_qnoise.add_vline(x=right_f,line_dash='dash',line_color='green')
    fig_qnoise.add_vline(x=left_f,line_dash='dash',line_color='green')
    fig_qnoise.update_layout(title="ASD (qnoised, qshot)",xaxis_title="Frequency [Hz]",yaxis_title="Sensitivity [1/rt Hz]")
    fig_qnoise.show()
    
    fig_signal = go.Figure()
    fig_signal.add_trace(go.Scatter(x=fsig, y=np.abs(out['signal']),mode='lines+markers'))
    fig_signal.update_xaxes(type="log")
    fig_signal.update_yaxes(type="log")
    fig_signal.add_vline(x=peak_f)
    fig_signal.add_vline(x=right_f,line_dash='dash',line_color='green')
    fig_signal.add_vline(x=left_f,line_dash='dash',line_color='green')
    fig_signal.update_layout(title="Signal Gain (pd1)",xaxis_title="Frequency [Hz]",yaxis_title="Power [W]")
    fig_signal.show()
    
    # Print model outputs
    print(f"Optimal prmT: {prmT}")
    print(f"Peak Sensitivity: {peak_sens} 1/rt Hz, Peak Frequency: {peak_f}Hz, Peak FWHM: {peak_bw}Hz")
    print(f"Input laser power: {np.max(np.abs(out['P_in']))}W")
    print(f"PRC power: {np.max(np.abs(out['P_PRC']))*1e-3}kW")
    print(f"Laser power incident on BS: {np.max(np.abs(out['P_BS']))*1e-3}kW")
    print(f"X-arm cavity power: {np.max(np.abs(out['P_armX']))*1e-6}MW")
    print(f"Y-arm cavity power: {np.max(np.abs(out['P_armY']))*1e-6}MW")
    print(f"SRC power: {np.max(np.abs(out['P_SRC']))}W")
    
    def save_curve(arg):
        lines = [f"{fsig[i]} {np.abs(out['NSR_with_RP'])[i]}\n" for i in range(201)]
        lines[-1].rstrip('\n')
        filename = f"vSRM_{phiComm}_{phiDiff}_{srcL}_{itmT}_{prmT}_{lasPow}_ASD_with_RP.txt"
        file = open(save_path+filename, "w")
        file.writelines(lines)
        file.close()
        print(f"Saved {filename}!")

    button_save = widgets.Button(description = 'Save')   
    button_save.on_click(save_curve)
    display(button_save)

# Default values here
widgets.interact_manual(plot, phiComm=widgets.Text(value=f"{default_phiComm}"),
                 phiDiff=widgets.Text(value=f"{default_phiDiff}"), srcL=widgets.Text(value=f"{default_srcL}"), itmT=widgets.Text(value=f"{default_itmT}"));

# Auto-tune prmT with findpeaks to maximise arm cavity power (impedance matching)
def find_optimal_prmT(phiComm,phiDiff,srcL,itmT):
    armPow = 4.5e6
    lasPow = 500
    
    vary_prmT = np.geomspace(0.01,0.5,100)
    circX = np.zeros((100,))
    # Find prmT to maximise arm cavity power
    for i, prmT in enumerate(vary_prmT):
        kat = finesse.Model()
        kat.parse(
        f"""
        # NEMO Base Model (simplified from OzHF_ITMloss_4km.kat and converted to Finesse 3)
        ###########################################################################
        ###   Variables
        ###########################################################################
        var Larm 4000
        var Mtm  74.1 # from NEMO paper (94.4 in OzHF_ITMloss_4km.kat)
        var itmT {itmT}
        var lmichx 4.5
        var lmichy 4.45

        ###########################################################################
        ###   Input optics
        ###########################################################################
        l L0 500

        s l_in L0.p1 prm.p1
        # Power recycling mirror
        m prm T={prmT} L=2e-05 phi=90
        s prc prm.p2 bs.p1 L=53

        # Central beamsplitter
        bs bs R=0.4999625 T=0.4999625 alpha=45

        # CHECK Input laser power
        # pd P_in L0.p1.o
        # CHECK Laser power incident on BS
        # pd P_BS bs.p1.i
        # CHECK PRC Power
        # pd P_PRC bs.p1.o

        ###########################################################################
        ###   X arm
        ###########################################################################
        s lx bs.p3 itmxar.p1 L=lmichx

        m itmxar T=1-265.0e-06 L=265.0e-06 phi=180 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
        s ar_thick itmxar.p2 itmx.p1 L=0
        m itmx T=itmT L=20u phi=180
        s LX itmx.p2 etmx.p1 L=Larm

        m etmx T=5u L=20u phi=179.99999

        pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
        pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

        # CHECK X-arm cavity power
        pd P_armX etmx.p1.i

        ###########################################################################
        ###   Y arm
        ###########################################################################
        s ly bs.p2 itmyar.p1 L=lmichy

        m itmyar T=1-265.0e-06 L=265.0e-06 phi=90 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
        s ar_thicky itmyar.p2 itmy.p1 L=0
        m itmy T=itmT L=20u phi=90
        s LY itmy.p2 etmy.p1 L=Larm

        m etmy T=5u L=20u phi=90.00001

        pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
        pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

        # CHECK Y-arm cavity power
        # pd P_armY etmy.p1.i

        ###########################################################################
        ###   vSRM
        ###########################################################################
        s src bs.p4 SRC_BS.p1 L={srcL}
        bs SRC_BS T=0.5 L=0 alpha=45
        s vSRC1 SRC_BS.p2 vSRM1.p1 L=4.5
        m vSRM1 T=0 L=0 phi={-90+phiComm+phiDiff}
        s vSRC2 SRC_BS.p3 vSRM2.p1 L=4.5
        m vSRM2 T=0 L=0 phi={0+phiComm-phiDiff}

        # CHECK SRC power
        # pd P_SRC SRC_BS.p1.i

        ###########################################################################
        ###   Output & squeezing (from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb)
        ###########################################################################
        noxaxis()
        """
        )
        out = kat.run()
        circX[i] = out['P_armX']
    
    peak_power = circX[np.argmax(circX)]
    peak_T = vary_prmT[np.argmax(circX)]
    
    if peak_power < armPow and lasPow < 500*1.10:
        while peak_power < armPow:
            lasPow += 1
            kat = finesse.Model()
            kat.parse(
            f"""
            # NEMO Base Model (simplified from OzHF_ITMloss_4km.kat and converted to Finesse 3)
            ###########################################################################
            ###   Variables
            ###########################################################################
            var Larm 4000
            var Mtm  74.1 # from NEMO paper (94.4 in OzHF_ITMloss_4km.kat)
            var itmT {itmT}
            var lmichx 4.5
            var lmichy 4.45

            ###########################################################################
            ###   Input optics
            ###########################################################################
            l L0 {lasPow}

            s l_in L0.p1 prm.p1
            # Power recycling mirror
            m prm T={peak_T} L=2e-05 phi=90
            s prc prm.p2 bs.p1 L=53

            # Central beamsplitter
            bs bs R=0.4999625 T=0.4999625 alpha=45

            # CHECK Input laser power
            # pd P_in L0.p1.o
            # CHECK Laser power incident on BS
            # pd P_BS bs.p1.i
            # CHECK PRC Power
            # pd P_PRC bs.p1.o

            ###########################################################################
            ###   X arm
            ###########################################################################
            s lx bs.p3 itmxar.p1 L=lmichx

            m itmxar T=1-265.0e-06 L=265.0e-06 phi=180 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
            s ar_thick itmxar.p2 itmx.p1 L=0
            m itmx T=itmT L=20u phi=180
            s LX itmx.p2 etmx.p1 L=Larm

            m etmx T=5u L=20u phi=179.99999

            pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
            pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

            # CHECK X-arm cavity power
            pd P_armX etmx.p1.i

            ###########################################################################
            ###   Y arm
            ###########################################################################
            s ly bs.p2 itmyar.p1 L=lmichy

            m itmyar T=1-265.0e-06 L=265.0e-06 phi=90 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
            s ar_thicky itmyar.p2 itmy.p1 L=0
            m itmy T=itmT L=20u phi=90
            s LY itmy.p2 etmy.p1 L=Larm

            m etmy T=5u L=20u phi=90.00001

            pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
            pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

            # CHECK Y-arm cavity power
            # pd P_armY etmy.p1.i

            ###########################################################################
            ###   vSRM
            ###########################################################################
            s src bs.p4 SRC_BS.p1 L={srcL}
            bs SRC_BS T=0.5 L=0 alpha=45
            s vSRC1 SRC_BS.p2 vSRM1.p1 L=4.5
            m vSRM1 T=0 L=0 phi={-90+phiComm+phiDiff}
            s vSRC2 SRC_BS.p3 vSRM2.p1 L=4.5
            m vSRM2 T=0 L=0 phi={0+phiComm-phiDiff}

            # CHECK SRC power
            # pd P_SRC SRC_BS.p1.i

            ###########################################################################
            ###   Output & squeezing (from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb)
            ###########################################################################
            noxaxis()
            """
            )
            out = kat.run()
            peak_power = out['P_armX']
    else: 
        while peak_power > armPow:
            lasPow -= 1
            kat = finesse.Model()
            kat.parse(
            f"""
            # NEMO Base Model (simplified from OzHF_ITMloss_4km.kat and converted to Finesse 3)
            ###########################################################################
            ###   Variables
            ###########################################################################
            var Larm 4000
            var Mtm  74.1 # from NEMO paper (94.4 in OzHF_ITMloss_4km.kat)
            var itmT {itmT}
            var lmichx 4.5
            var lmichy 4.45

            ###########################################################################
            ###   Input optics
            ###########################################################################
            l L0 {lasPow}

            s l_in L0.p1 prm.p1
            # Power recycling mirror
            m prm T={peak_T} L=2e-05 phi=90
            s prc prm.p2 bs.p1 L=53

            # Central beamsplitter
            bs bs R=0.4999625 T=0.4999625 alpha=45

            # CHECK Input laser power
            # pd P_in L0.p1.o
            # CHECK Laser power incident on BS
            # pd P_BS bs.p1.i
            # CHECK PRC Power
            # pd P_PRC bs.p1.o

            ###########################################################################
            ###   X arm
            ###########################################################################
            s lx bs.p3 itmxar.p1 L=lmichx

            m itmxar T=1-265.0e-06 L=265.0e-06 phi=180 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
            s ar_thick itmxar.p2 itmx.p1 L=0
            m itmx T=itmT L=20u phi=180
            s LX itmx.p2 etmx.p1 L=Larm

            m etmx T=5u L=20u phi=179.99999

            pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
            pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

            # CHECK X-arm cavity power
            pd P_armX etmx.p1.i

            ###########################################################################
            ###   Y arm
            ###########################################################################
            s ly bs.p2 itmyar.p1 L=lmichy

            m itmyar T=1-265.0e-06 L=265.0e-06 phi=90 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
            s ar_thicky itmyar.p2 itmy.p1 L=0
            m itmy T=itmT L=20u phi=90
            s LY itmy.p2 etmy.p1 L=Larm

            m etmy T=5u L=20u phi=90.00001

            pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
            pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

            # CHECK Y-arm cavity power
            # pd P_armY etmy.p1.i

            ###########################################################################
            ###   vSRM
            ###########################################################################
            s src bs.p4 SRC_BS.p1 L={srcL}
            bs SRC_BS T=0.5 L=0 alpha=45
            s vSRC1 SRC_BS.p2 vSRM1.p1 L=4.5
            m vSRM1 T=0 L=0 phi={-90+phiComm+phiDiff}
            s vSRC2 SRC_BS.p3 vSRM2.p1 L=4.5
            m vSRM2 T=0 L=0 phi={0+phiComm-phiDiff}

            # CHECK SRC power
            # pd P_SRC SRC_BS.p1.i

            ###########################################################################
            ###   Output & squeezing (from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb)
            ###########################################################################
            noxaxis()
            """
            )
            out = kat.run()
            peak_power = out['P_armX']
        lasPow += 1
    
    return peak_power, peak_T, lasPow

interactive(children=(Text(value='0', continuous_update=False, description='phiComm'), Text(value='6.3565', co…

# Section 2. One-Parameter Exploration  
We focus our attention on the vSRM configuration as it provides a practical solution to tuning the frequency response between observing runs. During one-parameter sweeps, all other parameters are held at their default values. Some cells may take a few minutes to execute. The parameters we investigate are:  
1. Common-mode tuning (move vSRM mirrors in the same direction) $\phi_\text{commm}$

2. Differential-mode tuning (move vSRM mirrors in opposite directions) $\phi_\text{diff}$

3. SRC length $L_\text{SRC}$

4. ITM transmittivity $T_\text{ITM}$  

All plots are made using plottly and are interactive! Have a play around to get some intuition about parameter space (as I did). Note that all the curves generated below do not include squeezing (simply add ``squeezing=True`` to any call to the function ``Finesse_sensitivity_into_Bilby`` to add squeezing).

In [14]:
import nemo_optimisation_modules as nom
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np
pio.renderers.default='iframe'
# pio.renderers.default='notebook_connected'

default_phiComm = 0
default_phiDiff = 6.3565
default_srcL = 354
default_itmT = 0.01397

## 2.1 Example Sensitivity Curves  
For each parameter the cells below will overlay five curves corresponding to different parameter values. The key observation here is that four-dimensional parameter space is very rich and there are many trade-offs e.g. higher sensitivity but lower bandwidth. It is not obvious a priori that extremising multiple parameters simultaneously will lead to an optimal design.

### 2.1.1 $\phi_\text{comm}$  

In [15]:
vary_phiComm = [default_phiComm,1,3,6,10]

fig_qnoise = go.Figure()
fig_qnoise.update_xaxes(type="log")
fig_qnoise.update_yaxes(type="log")
fig_qnoise.update_layout(title="Common-Mode Tuning (phiComm) Example Curves",xaxis_title="Frequency [Hz]",yaxis_title="ASD (qnoised) [1/rt Hz]")

changed_itmT = True
store_prmT = 0
store_lasPow = 0
    
for phiComm in vary_phiComm:
    if changed_itmT:
        fsig, ASDarr, prmT, lasPow = nom.Finesse_sensitivity_into_Bilby(phiComm,default_phiDiff,default_srcL,default_itmT,prmT=0,lasPow=0,optimise_prmT=True,squeezing=False)
        print(f"CHECK phiComm: {phiComm}, Optimal prmT: {prmT}, Laser Power: {lasPow}")
        fig_qnoise.add_trace(go.Scatter(x=fsig, y=ASDarr,mode='lines+markers',name=f"phiComm={phiComm}"))
        store_prmT = prmT
        store_lasPow = lasPow
        changed_itmT = False
    else: 
        fsig, ASDarr, prmT, lasPow = nom.Finesse_sensitivity_into_Bilby(phiComm,default_phiDiff,default_srcL,default_itmT,prmT=store_prmT,lasPow=store_lasPow,optimise_prmT=False,squeezing=False)
        print(f"CHECK phiComm: {phiComm}, Optimal prmT: {prmT}, Laser Power: {lasPow}")
        fig_qnoise.add_trace(go.Scatter(x=fsig, y=ASDarr,mode='lines+markers',name=f"phiComm={phiComm}"))
        
fig_qnoise.show()

CHECK phiComm: 0, Optimal prmT: 0.013186489467542296, Laser Power: 428
CHECK phiComm: 1, Optimal prmT: 0.013186489467542296, Laser Power: 428.0
CHECK phiComm: 3, Optimal prmT: 0.013186489467542296, Laser Power: 428.0
CHECK phiComm: 6, Optimal prmT: 0.013186489467542296, Laser Power: 428.0
CHECK phiComm: 10, Optimal prmT: 0.013186489467542296, Laser Power: 428.0


### 2.1.2 $\phi_\text{diff}$

In [16]:
vary_phiDiff = [1,2,4,default_phiDiff,10]

fig_qnoise = go.Figure()
fig_qnoise.update_xaxes(type="log")
fig_qnoise.update_yaxes(type="log")
fig_qnoise.update_layout(title="Differential-Mode Tuning (phiDiff) Example Curves",xaxis_title="Frequency [Hz]",yaxis_title="ASD (qnoised) [1/rt Hz]")
    
changed_itmT = True
store_prmT = 0
store_lasPow = 0
    
for phiDiff in vary_phiDiff:
    if changed_itmT:
        fsig, ASDarr, prmT, lasPow = nom.Finesse_sensitivity_into_Bilby(default_phiComm,phiDiff,default_srcL,default_itmT,prmT=0,lasPow=0,optimise_prmT=True,squeezing=False)
        print(f"CHECK phiDiff: {phiDiff}, Optimal prmT: {prmT}, Laser Power: {lasPow}")
        fig_qnoise.add_trace(go.Scatter(x=fsig, y=ASDarr,mode='lines+markers',name=f"phiDiff={phiDiff}"))
        store_prmT = prmT
        store_lasPow = lasPow
        changed_itmT = False
    else: 
        fsig, ASDarr, prmT, lasPow = nom.Finesse_sensitivity_into_Bilby(default_phiComm,phiDiff,default_srcL,default_itmT,prmT=store_prmT,lasPow=store_lasPow,optimise_prmT=False,squeezing=False)
        print(f"CHECK phiDiff: {phiDiff}, Optimal prmT: {prmT}, Laser Power: {lasPow}")
        fig_qnoise.add_trace(go.Scatter(x=fsig, y=ASDarr,mode='lines+markers',name=f"phiDiff={phiDiff}"))
        
fig_qnoise.show()

CHECK phiDiff: 1, Optimal prmT: 0.013186489467542296, Laser Power: 428
CHECK phiDiff: 2, Optimal prmT: 0.013186489467542296, Laser Power: 428.0
CHECK phiDiff: 4, Optimal prmT: 0.013186489467542296, Laser Power: 428.0
CHECK phiDiff: 6.3565, Optimal prmT: 0.013186489467542296, Laser Power: 428.0
CHECK phiDiff: 10, Optimal prmT: 0.013186489467542296, Laser Power: 428.0


### 2.1.3 $L_\text{SRC}$

In [17]:
vary_srcL = [50,150,default_srcL,600,1000]

fig_qnoise = go.Figure()
fig_qnoise.update_xaxes(type="log")
fig_qnoise.update_yaxes(type="log")
fig_qnoise.update_layout(title="SRC Length (srcL) Example Curves",xaxis_title="Frequency [Hz]",yaxis_title="ASD (qnoised) [1/rt Hz]")

changed_itmT = True
store_prmT = 0
store_lasPow = 0

for srcL in vary_srcL:
    if changed_itmT:
        fsig, ASDarr, prmT, lasPow = nom.Finesse_sensitivity_into_Bilby(default_phiComm,default_phiDiff,srcL,default_itmT,prmT=0,lasPow=0,optimise_prmT=True,squeezing=False)
        print(f"CHECK srcL: {srcL}, Optimal prmT: {prmT}, Laser Power: {lasPow}")
        fig_qnoise.add_trace(go.Scatter(x=fsig, y=ASDarr,mode='lines+markers',name=f"srcL={srcL}"))
        store_prmT = prmT
        store_lasPow = lasPow
        changed_itmT = False
    else:
        fsig, ASDarr, prmT, lasPow = nom.Finesse_sensitivity_into_Bilby(default_phiComm,default_phiDiff,srcL,default_itmT,prmT=store_prmT,lasPow=store_lasPow,optimise_prmT=False,squeezing=False)
        print(f"CHECK srcL: {srcL}, Optimal prmT: {prmT}, Laser Power: {lasPow}")
        fig_qnoise.add_trace(go.Scatter(x=fsig, y=ASDarr,mode='lines+markers',name=f"srcL={srcL}"))
fig_qnoise.show()

CHECK srcL: 50, Optimal prmT: 0.013186489467542296, Laser Power: 428
CHECK srcL: 150, Optimal prmT: 0.013186489467542296, Laser Power: 428.0
CHECK srcL: 354, Optimal prmT: 0.013186489467542296, Laser Power: 428.0
CHECK srcL: 600, Optimal prmT: 0.013186489467542296, Laser Power: 428.0
CHECK srcL: 1000, Optimal prmT: 0.013186489467542296, Laser Power: 428.0


### 2.1.4 $T_\text{ITM}$

In [18]:
vary_itmT = [0.005,0.01,default_itmT,0.02,0.05]

fig_qnoise = go.Figure()
fig_qnoise.update_xaxes(type="log")
fig_qnoise.update_yaxes(type="log")
fig_qnoise.update_layout(title="ITM Transmission (itmT) Example Curves",xaxis_title="Frequency [Hz]",yaxis_title="ASD (qnoised) [1/rt Hz]")

for itmT in vary_itmT:
    fsig, ASDarr, prmT, lasPow = nom.Finesse_sensitivity_into_Bilby(default_phiComm,default_phiDiff,default_srcL,itmT,prmT=0,lasPow=0,optimise_prmT=True,squeezing=False)
    print(f"CHECK itmT: {itmT}, Optimal prmT: {prmT}, Laser Power: {lasPow}")
    fig_qnoise.add_trace(go.Scatter(x=fsig, y=ASDarr,mode='lines+markers',name=f"itmT={itmT}"))
    
fig_qnoise.show()

CHECK itmT: 0.005, Optimal prmT: 0.03541294309885406, Laser Power: 414
CHECK itmT: 0.01, Optimal prmT: 0.018818326997598396, Laser Power: 422
CHECK itmT: 0.01397, Optimal prmT: 0.013186489467542296, Laser Power: 428
CHECK itmT: 0.02, Optimal prmT: 0.01, Laser Power: 438
CHECK itmT: 0.05, Optimal prmT: 0.01, Laser Power: 550


## 2.2 Sensitivity Curve vs. Parameter Surface (3D)  
For each parameter we can visualise how the sensitivity curve changes by plotting parameter vs. frequency vs. sensitivity.

### 2.2.1 $\phi_\text{comm}$  

In [19]:
vary_phiComm = np.geomspace(1e-1,180,101)
fsig = np.geomspace(100,10e3,201)
sens = np.zeros((101,201))
peak_sens_phiComm = np.zeros(101)
peak_f_phiComm = np.zeros(101)
peak_bw_phiComm = np.zeros(101)

changed_itmT = True
store_prmT = 0
store_lasPow = 0

for i, phiComm in enumerate(vary_phiComm):
    if changed_itmT:
        fsig, ASDarr, prmT, lasPow, peak_dict = nom.Finesse_sensitivity_into_Bilby(phiComm,default_phiDiff,default_srcL,default_itmT,prmT=0,lasPow=0,optimise_prmT=True,export_peaks=True)
        sens[i,:] = ASDarr
        store_prmT = prmT
        store_lasPow = lasPow
        changed_itmT = False
        peak_sens_phiComm[i] = peak_dict['peak_sens']
        peak_f_phiComm[i] = peak_dict['peak_f']
        peak_bw_phiComm[i] = peak_dict['peak_bw']
    else: 
        fsig, ASDarr, prmT, lasPow, peak_dict = nom.Finesse_sensitivity_into_Bilby(phiComm,default_phiDiff,default_srcL,default_itmT,store_prmT,store_lasPow,optimise_prmT=False,export_peaks=True)
        sens[i,:] = ASDarr
        peak_sens_phiComm[i] = peak_dict['peak_sens']
        peak_f_phiComm[i] = peak_dict['peak_f']
        peak_bw_phiComm[i] = peak_dict['peak_bw']

fig_surf = go.Figure(data=[go.Surface(x=fsig,y=vary_phiComm,z=np.log10(sens),opacity=0.5)])
fig_surf.update_layout(scene=dict(xaxis=dict(type='log',title="Frequency [Hz]"),
                             yaxis=dict(title="phiComm [deg]"),
                            zaxis=dict(title="log ASD (qnoised) [1/rt Hz]")))
fig_surf.update_layout(title='Sensitivity vs. phiComm')
fig_surf.show()

### 2.2.2 $\phi_\text{diff}$

In [20]:
vary_phiDiff = np.geomspace(1e-1,180-1e-1,101)  # At 180deg 
fsig = np.geomspace(100,10e3,201)
sens = np.zeros((101,201))
peak_sens_phiDiff = np.zeros(101)
peak_f_phiDiff = np.zeros(101)
peak_bw_phiDiff = np.zeros(101)

changed_itmT = True
store_prmT = 0
store_lasPow = 0

for i, phiDiff in enumerate(vary_phiDiff):
    if changed_itmT:
        fsig, ASDarr, prmT, lasPow, peak_dict = nom.Finesse_sensitivity_into_Bilby(default_phiComm,phiDiff,default_srcL,default_itmT,prmT=0,lasPow=0,optimise_prmT=True,export_peaks=True)
        sens[i,:] = ASDarr
        store_prmT = prmT
        store_lasPow = lasPow
        changed_itmT = False
        peak_sens_phiDiff[i] = peak_dict['peak_sens']
        peak_f_phiDiff[i] = peak_dict['peak_f']
        peak_bw_phiDiff[i] = peak_dict['peak_bw']
    else: 
        fsig, ASDarr, prmT, lasPow, peak_dict = nom.Finesse_sensitivity_into_Bilby(default_phiComm,phiDiff,default_srcL,default_itmT,store_prmT,store_lasPow,optimise_prmT=False,export_peaks=True)
        sens[i,:] = ASDarr
        peak_sens_phiDiff[i] = peak_dict['peak_sens']
        peak_f_phiDiff[i] = peak_dict['peak_f']
        peak_bw_phiDiff[i] = peak_dict['peak_bw']

fig_surf = go.Figure(data=[go.Surface(x=fsig,y=vary_phiDiff,z=np.log10(sens),opacity=0.5)])
fig_surf.update_layout(scene=dict(xaxis=dict(type='log',title="Frequency [Hz]"),
                             yaxis=dict(title="phiDiff [deg]"),
                            zaxis=dict(title="log ASD (qnoised) [1/rt Hz]")))
fig_surf.update_layout(title='Sensitivity vs. phiDiff')
fig_surf.show()

### 2.2.3 $L_\text{SRC}$

In [21]:
vary_srcL = np.geomspace(10,1e3,101)
fsig = np.geomspace(100,10e3,201)
sens = np.zeros((101,201))
peak_sens_srcL = np.zeros(101)
peak_f_srcL = np.zeros(101)
peak_bw_srcL = np.zeros(101)

changed_itmT = True
store_prmT = 0
store_lasPow = 0

for i, srcL in enumerate(vary_srcL):
    if changed_itmT:
        fsig, ASDarr, prmT, lasPow, peak_dict = nom.Finesse_sensitivity_into_Bilby(default_phiComm,default_phiDiff,srcL,default_itmT,prmT=0,lasPow=0,optimise_prmT=True,export_peaks=True)
        sens[i,:] = ASDarr
        store_prmT = prmT
        store_lasPow = lasPow
        changed_itmT = False
        peak_sens_srcL[i] = peak_dict['peak_sens']
        peak_f_srcL[i] = peak_dict['peak_f']
        peak_bw_srcL[i] = peak_dict['peak_bw']
    else: 
        fsig, ASDarr, prmT, lasPow, peak_dict = nom.Finesse_sensitivity_into_Bilby(default_phiComm,default_phiDiff,srcL,default_itmT,store_prmT,store_lasPow,optimise_prmT=False,export_peaks=True)
        sens[i,:] = ASDarr
        peak_sens_srcL[i] = peak_dict['peak_sens']
        peak_f_srcL[i] = peak_dict['peak_f']
        peak_bw_srcL[i] = peak_dict['peak_bw']

fig_surf = go.Figure(data=[go.Surface(x=fsig,y=vary_srcL,z=np.log10(sens),opacity=0.5)])
fig_surf.update_layout(scene=dict(xaxis=dict(type='log',title="Frequency [Hz]"),
                             yaxis=dict(title="srcL [m]"),
                            zaxis=dict(title="log ASD (qnoised) [1/rt Hz]")))
fig_surf.update_layout(title='Sensitivity vs. srcL')
fig_surf.show()

### 2.2.4 $T_\text{ITM}$  
When $T_\text{ITM}$ is changed, $T_\text{PRM}$ is re-calculated to meet the impedance-matching condition i.e. the arm circulating power is maximised. Afterwards, the input laser power is increased from 500W (up to 550W at most) or decreased until the nominal value for arm circulating power (4.5MW) is reached. The colour on the first surface plot is $T_\text{PRM}$, the second is the laser power. This process takes a long time so the generating code is commented out (use load and find the associated numpy array in ``./SaveArrays/``).  
**Observations**: See ``SaveArrays/SaveArrays/Sec2.2.4_store_prmT.npy`` that for many $T_\text{ITM}>0.01$, the impedance-matched $T_\text{PRM}$ saturates at 0.01 (and is the case for the preliminary optimal detector design in Section 4). We subsequently decrease the lower limit from ``1e-2`` to ``1e-3``.

In [22]:
vary_itmT = np.geomspace(1e-3,0.5,101)
fsig = np.geomspace(100,10e3,201)
# sens = np.zeros((101,201))
# store_prmT = np.zeros((101,201))
# store_lasPow = np.zeros((101,201))
# peak_sens_itmT = np.zeros(101)
# peak_f_itmT = np.zeros(101)
# peak_bw_itmT = np.zeros(101)

# for i, itmT in enumerate(vary_itmT):
#         fsig, ASDarr, prmT, lasPow, peak_dict = nom.Finesse_sensitivity_into_Bilby(default_phiDiff,default_phiComm,default_srcL,itmT,prmT=0,lasPow=0,optimise_prmT=True,export_peaks=True)
#         sens[i,:] = ASDarr
#         store_prmT[i,:] = np.repeat(prmT, 201)
#         store_lasPow[i,:] = np.repeat(lasPow, 201)
#         peak_sens_itmT[i] = peak_dict['peak_sens']
#         peak_f_itmT[i] = peak_dict['peak_f']
#         peak_bw_itmT[i] = peak_dict['peak_bw']

# np.save("SaveArrays/Sec2.2.4_store_prmT_lowerLim.npy",store_prmT)
# np.save("SaveArrays/Sec2.2.4_store_sens_lowerLim.npy",sens)
# np.save("SaveArrays/Sec2.2.4_store_peak_sens_itmT_lowerLim.npy",peak_sens_itmT)
# np.save("SaveArrays/Sec2.2.4_store_peak_f_itmT_lowerLim.npy",peak_f_itmT)
# np.save("SaveArrays/Sec2.2.4_store_peak_bw_itmT_lowerLim.npy",peak_bw_itmT)
# np.save("SaveArrays/Sec2.2.4_store_lasPow_lowerLim.npy",store_lasPow)

sens = np.load("SaveArrays/Sec2.2.4_store_sens_lowerLim.npy")
store_prmT = np.load("SaveArrays/Sec2.2.4_store_prmT_lowerLim.npy")
store_lasPow = np.load("SaveArrays/Sec2.2.4_store_lasPow_lowerLim.npy")
peak_sens_itmT = np.load("SaveArrays/Sec2.2.4_store_peak_sens_itmT_lowerLim.npy")
peak_f_itmT = np.load("SaveArrays/Sec2.2.4_store_peak_f_itmT_lowerLim.npy")
peak_bw_itmT = np.load("SaveArrays/Sec2.2.4_store_peak_bw_itmT_lowerLim.npy")

fig_sens = go.Figure(data=[go.Surface(x=fsig,y=vary_itmT,z=np.log10(sens),opacity=0.5)])
fig_sens.update_layout(scene=dict(xaxis=dict(type='log',title="Frequency [Hz]"),yaxis=dict(title="itmT"),zaxis=dict(title="log ASD (qnoised) [1/rt Hz]")))
fig_sens.update_layout(title='Sensitivity vs. itmT')
fig_sens.show()

fig_prmT = go.Figure(data=[go.Surface(x=fsig,y=vary_itmT,z=store_prmT,opacity=0.5)])
fig_prmT.update_layout(scene=dict(xaxis=dict(type='log',title="Frequency [Hz]"),yaxis=dict(title="itmT"),zaxis=dict(title="prmT")))
fig_prmT.update_layout(title='prmT vs. itmT')
fig_prmT.show()

fig_lasPow = go.Figure(data=[go.Surface(x=fsig,y=vary_itmT,z=store_lasPow,opacity=0.5)])
fig_lasPow.update_layout(scene=dict(xaxis=dict(type='log',title="Frequency [Hz]"),yaxis=dict(title="itmT"),zaxis=dict(title="lasPow [W]")))
fig_lasPow.update_layout(title='lasPow vs. itmT')
fig_lasPow.show()

## 2.3 Peak Properties vs. Parameter
Using the negative ASD curve, scipy's standard ``findpeaks`` function is used to find the *first peak* (if it exists). The bandwidth is calculated as the full width at half maximum using scipy's ``peak_widths`` function. If no peak is found, no data is plotted (hence, the gaps in some plots). The half maximum frequencies are interpolated on the log frequency scale using scipy's ``interp1d`` function. Each of these peak properties is plotted against the parameter. The ideal peak has a peak frequency in the band 2-4kHz (containing the most import post-merger spectral information), high peak sensitivity (small ASD), and high bandwidth. Note that the parameters are log-spaced (favouring higher resolution near small values near the default NEMO parameters). 

**Note**: Run Section 2.2.X to get peak data for 2.3.X first! (the below cells are purely plotting functions.)

### 2.3.1 $\phi_\text{comm}$  

In [25]:
fig_peak_sens = go.Figure()
fig_peak_sens.add_trace(go.Scatter(x=vary_phiComm, y=peak_sens_phiComm,mode='lines+markers',name='Peak Frequency'))
fig_peak_sens.update_layout(title="Peak Sensitivity vs. phiComm",xaxis_title="phiComm [deg]",yaxis_title="Peak Sensitivity [1/rt Hz]")
fig_peak_sens.show()

fig_peak_f = go.Figure()
fig_peak_f.add_trace(go.Scatter(x=vary_phiComm, y=peak_f_phiComm,mode='lines+markers',name='Peak'))
fig_peak_f.update_layout(title="Peak Frequency vs. phiComm",xaxis_title="phiComm [deg]",yaxis_title="Peak Frequency [Hz]")
fig_peak_f.show()

fig_peak_bw = go.Figure()
fig_peak_bw.add_trace(go.Scatter(x=vary_phiComm, y=peak_bw_phiComm,mode='lines+markers',name='Peak'))
fig_peak_bw.update_layout(title="Peak Bandwidth vs. phiComm",xaxis_title="phiComm [deg]",yaxis_title="Peak Bandwidth [Hz]")
fig_peak_bw.show()

### 2.3.2 $\phi_\text{diff}$

In [26]:
fig_peak_sens = go.Figure()
fig_peak_sens.add_trace(go.Scatter(x=vary_phiDiff, y=peak_sens_phiDiff,mode='lines+markers',name='Peak Frequency'))
fig_peak_sens.update_layout(title="Peak Sensitivity vs. phiDiff",xaxis_title="phiDiff [deg]",yaxis_title="Peak Sensitivity [1/rt Hz]")
fig_peak_sens.show()

fig_peak_f = go.Figure()
fig_peak_f.add_trace(go.Scatter(x=vary_phiDiff, y=peak_f_phiDiff,mode='lines+markers',name='Peak'))
fig_peak_f.update_layout(title="Peak Frequency vs. phiDiff",xaxis_title="phiDiff [deg]",yaxis_title="Peak Frequency [Hz]")
fig_peak_f.show()

fig_peak_bw = go.Figure()
fig_peak_bw.add_trace(go.Scatter(x=vary_phiDiff, y=peak_bw_phiDiff,mode='lines+markers',name='Peak'))
fig_peak_bw.update_layout(title="Peak Bandwidth vs. phiDiff",xaxis_title="phiDiff [deg]",yaxis_title="Peak Bandwidth [Hz]")
fig_peak_bw.show()

### 2.3.3 $L_\text{SRC}$

In [27]:
fig_peak_sens = go.Figure()
fig_peak_sens.add_trace(go.Scatter(x=vary_srcL, y=peak_sens_srcL,mode='lines+markers',name='Peak Frequency'))
fig_peak_sens.update_layout(title="Peak Sensitivity vs. srcL",xaxis_title="srcL [deg]",yaxis_title="Peak Sensitivity [1/rt Hz]")
fig_peak_sens.show()

fig_peak_f = go.Figure()
fig_peak_f.add_trace(go.Scatter(x=vary_srcL, y=peak_f_srcL,mode='lines+markers',name='Peak'))
fig_peak_f.update_layout(title="Peak Frequency vs. srcL",xaxis_title="srcL [deg]",yaxis_title="Peak Frequency [Hz]")
fig_peak_f.show()

fig_peak_bw = go.Figure()
fig_peak_bw.add_trace(go.Scatter(x=vary_srcL, y=peak_bw_srcL,mode='lines+markers',name='Peak'))
fig_peak_bw.update_layout(title="Peak Bandwidth vs. srcL",xaxis_title="srcL [deg]",yaxis_title="Peak Bandwidth [Hz]")
fig_peak_bw.show()

### 2.3.4 $T_\text{ITM}$  
**TODO**: Determine why the curves are significantly not as smooth as for the other parameters (including at high resolution).

In [28]:
fig_peak_sens = go.Figure()
fig_peak_sens.add_trace(go.Scatter(x=vary_itmT, y=peak_sens_itmT,mode='lines+markers',name='Peak Frequency'))
fig_peak_sens.update_layout(title="Peak Sensitivity vs. itmT",xaxis_title="itmT",yaxis_title="Peak Sensitivity [1/rt Hz]")
fig_peak_sens.show()

fig_peak_f = go.Figure()
fig_peak_f.add_trace(go.Scatter(x=vary_itmT, y=peak_f_itmT,mode='lines+markers',name='Peak'))
fig_peak_f.update_layout(title="Peak Frequency vs. itmT",xaxis_title="itmT",yaxis_title="Peak Frequency [Hz]")
fig_peak_f.show()

fig_peak_bw = go.Figure()
fig_peak_bw.add_trace(go.Scatter(x=vary_itmT, y=peak_bw_itmT,mode='lines+markers',name='Peak'))
fig_peak_bw.update_layout(title="Peak Bandwidth vs. itmT",xaxis_title="itmT",yaxis_title="Peak Bandwidth [Hz]")
fig_peak_bw.show()

# Section 3. vSRM Tuning (Two-Parameter) Exploration  
In practice, it is difficult to alter SRC length and the ITM transmission after the construction of a detector. We focus now on co-varying the practically-tunable degrees of freedom i.e. the microscopic tuning of the vSRM in the common and differential modes. When there is detuning i.e. $\phi_{comm}\neq 0^\circ$ (assuming dark fringe condition on vSRM), radiation pressure makes the test masses respond to forces as though they were connected to an optical spring, leading to a drastic change in the shape of the sensitivity curve [Buonanno & Chen 2002]. This is because the SRM reflects quantum noise back into the interferometer, inducing a dynamical correlation between radiation pressure noise and quantum shot noise. This may enable the detector to beat the standard quantum limit (where radiation pressure and shot noise are equal). A priori, it is plausible that optimal detector designs may feature a detuned SRC.

In [29]:
import finesse
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import nemo_optimisation_modules as nom
import plotly.io as pio

## 3.1 SRM-vSRM Relations  
We show that an isolated Michelson acts as a variable-reflectivity mirror, with the reflectivity depending only on the differential-mode tuning. This also holds for a Michelson as a signal-recycling mirror. In the vSRM configuration, we make the following assumptions: mirrors are perfectly reflective ($T=0$), macroscopic arm length is fixed at 4.5m, and the dark fringe condition.

### 3.1.1 Isolated Michelson Reflectivity  
Refer to pp. 59 of de Vine's Honour's thesis. It can be shown that a Michelson has complex transmittivity and reflectivity $t_\text{Mich}=i\sin(2\omega\Delta L/c)e^{2i\omega L/c},r=\cos(2\omega\Delta L/c)e^{2i\omega L/c}$ where $\omega L/c$ is the phase acquired when a beam travels from the beamsplitter to the arm mirrors. We refer to $\phi_\text{diff}:=\omega\Delta L/c$ due to differential arm length. For the power transmittivity, it follows that $T_\text{SRM}(\phi_\text{diff})=1-\cos^2(2\phi_\text{diff}\cdot\frac{\pi}{180})$. (We assume the dark fringe condition but the result still holds for a translated cosine).

In [30]:
vary_diff = np.linspace(-180, 180, 361)
a_refl_abs = np.zeros((361,))
cosine_abs = np.abs(np.cos(2*np.deg2rad(vary_diff)))

for i, diff in enumerate(vary_diff):
    kat = finesse.Model()
    kat.parse(
        f"""
        ###########################################################################
        ###   Input optics
        ###########################################################################
        l L0 500

        ###########################################################################
        ###   vSRM
        ###########################################################################

        s src L0.p1 SRC_BS.p1 L=354
        bs SRC_BS T=0.5 L=0 alpha=45
        s vSRC1 SRC_BS.p2 vSRM1.p1 L=0
        m vSRM1 T=0 L=0 phi={90+diff}
        s vSRC2  SRC_BS.p3 vSRM2.p1 L=0
        m vSRM2 T=0 L=0 phi={0-diff}

        ad E_in L0.p1.o f=L0.f
        ad E_out SRC_BS.p1.o f=L0.f

        noxaxis()

        """
        )
    out = kat.run()
    E_in = out['E_in']
    E_out = out['E_out']
    a_refl_abs[i] = np.abs(E_out/E_in)
    
fig = go.Figure()
fig.update_layout(title="Michelson Reflectivity",xaxis_title="Differential-Mode Tuning [deg]",yaxis_title="|E_r/E_i|")
fig.add_trace(go.Scatter(x=vary_diff, y=a_refl_abs,mode='lines+markers',name='Finesse'))
fig.add_trace(go.Scatter(x=vary_diff, y=cosine_abs,mode='lines',name='Equation'))
fig.show()

### 3.1.2 Quantifying Error: Varying $\phi_\text{comm}$
In the SRM configuration, the signal-recycling cavity is formed between the SRM and the ITMs. For a vSRM, the SRC is formed between the vSRM mirrors and the ITMs i.e. main BS to ITMs + ``srcL`` + vSRM BS to vSRM mirrors. So ``srcL`` is not actually the SRC length (but it dominates the sum effectively). The vSRM common-mode tuning is related to the SRC tuning in the SRM configuration by $\phi_\text{SRC}=-90^\circ+\phi_\text{comm}$ (assuming a vSRM with the dark fringe condition). We make srcL equal and compare the error between ASD curves between vSRM and SRM configurations according to the SRM-vSRM relations. We skip impedance matching and squeezing for brevity (the results should still be indicative). It is more helpful to consider the relative error (with respect to the vSRM sensitivity) which is small (log scale).

In [31]:
vary_phiComm = np.linspace(-180, 180, 361)
rel_err = np.zeros((361,201))
abs_err = np.zeros((361,201))

vSRM_phiDiff = 6.3565
vSRM_srcL = 354
default_itmT = 0.01397
default_prmT = 0.03
default_lasPow = 500

SRM_srmT = 1-np.cos(2*vSRM_phiDiff*np.pi/180)**2
SRM_srcL = vSRM_srcL + 4.5

for i, phiComm in enumerate(vary_phiComm):
    # vSRM sensitivity
    fsig, ASDarr_vSRM, prmT, lasPow = nom.Finesse_sensitivity_into_Bilby(phiComm,vSRM_phiDiff,vSRM_srcL,default_itmT,prmT=default_prmT,lasPow=default_lasPow,optimise_prmT=False,export_peaks=False,squeezing=False)
    kat = finesse.Model()
    # SRM sensitivity (adjust for proper srcL)
    kat.parse(
    f"""
    # NEMO Base Model (simplified from OzHF_ITMloss_4km.kat and converted to Finesse 3)
    ###########################################################################
    ###   Variables
    ###########################################################################
    var Larm 4000
    var Mtm  74.1 # from NEMO paper (94.4 in OzHF_ITMloss_4km.kat)
    var itmT {default_itmT}
    var lmichx 4.5
    var lmichy 4.5 # Changed for symmetry

    ###########################################################################
    ###   Input optics
    ###########################################################################
    l L0 {default_lasPow}

    s l_in L0.p1 prm.p1
    # Power recycling mirror
    m prm T={default_prmT} L=2e-05 phi=90
    s prc prm.p2 bs.p1 L=53

    # Central beamsplitter
    bs bs R=0.4999625 T=0.4999625 alpha=45

    # CHECK Input laser power
    pd P_in L0.p1.o
    # CHECK Laser power incident on BS
    pd P_BS bs.p1.i
    # CHECK PRC Power
    pd P_PRC bs.p1.o

    ###########################################################################
    ###   X arm
    ###########################################################################
    s lx bs.p3 itmxar.p1 L=lmichx

    m itmxar T=1-265.0e-06 L=265.0e-06 phi=180 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
    s ar_thick itmxar.p2 itmx.p1 L=0
    m itmx T=itmT L=20u phi=180
    s LX itmx.p2 etmx.p1 L=Larm

    m etmx T=5u L=20u phi=179.99999

    pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
    pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

    # CHECK X-arm cavity power
    pd P_armX etmx.p1.i

    ###########################################################################
    ###   Y arm
    ###########################################################################
    s ly bs.p2 itmyar.p1 L=lmichy

    m itmyar T=1-265.0e-06 L=265.0e-06 phi=90 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
    s ar_thicky itmyar.p2 itmy.p1 L=0
    m itmy T=itmT L=20u phi=90
    s LY itmy.p2 etmy.p1 L=Larm

    m etmy T=5u L=20u phi=90.00001

    pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
    pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

    # CHECK Y-arm cavity power
    pd P_armY etmy.p1.i

    ###########################################################################
    ###   SRM
    ###########################################################################
    s src bs.p4 srm.p1 L={SRM_srcL}
    m srm R={0.99985-SRM_srmT} T={SRM_srmT} phi={-90+phiComm}

    # CHECK SRC power
    pd P_SRC srm.p1.i

    ###########################################################################
    ###   Output & squeezing (from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb)
    ###########################################################################
    dbs OFI 
    link(srm.p2, OFI.p1)
    readout_dc AS OFI.p3.o

    # A squeezed source could be injected into the dark port
    # sq sqz db=7 angle=90
    # link(sqz, OFI.p2)

    # ------------------------------------------------------------------------------
    # Degrees of Freedom
    # ------------------------------------------------------------------------------
    dof STRAIN LX.dofs.h +1  LY.dofs.h -1

    # signal generator
    sgen sig STRAIN

    qnoised NSR_with_RP AS.p1.i nsr=True
    # qshot NSR_without_RP AS.p1.i nsr=True
    # pd1 signal AS.p1.i f=fsig

    fsig(1)
    xaxis(fsig, log, 100, 10k, 200)
    """
    )
    out = kat.run()
    ASDarr_SRM = np.abs(out['NSR_with_RP'])
    abs_err[i,:] = np.abs(ASDarr_SRM-ASDarr_vSRM)
    rel_err[i,:] = np.abs((ASDarr_SRM-ASDarr_vSRM)/ASDarr_vSRM)

    
fig_abs_err = go.Figure(data=go.Contour(z=np.log10(abs_err),x=fsig,y=vary_phiComm))
fig_abs_err.update_layout(title="SRM-vSRM Relations Absolute ASD Error (qnoised)",xaxis_title="Frequency [Hz]",yaxis_title="phiComm [deg]")
fig_abs_err.update_xaxes(type="log")
fig_abs_err.show()

fig_rel_err = go.Figure(data=go.Contour(z=np.log10(rel_err),x=fsig,y=vary_phiComm))
fig_rel_err.update_layout(title="SRM-vSRM Relations Relative ASD Error (qnoised)",xaxis_title="Frequency [Hz]",yaxis_title="phiComm [deg]")
fig_rel_err.update_xaxes(type="log")
fig_rel_err.show()

### 3.1.3 Quantifying Error: Varying $\phi_\text{diff}$  
**TODO**: Why for $45^\circ<\phi_\text{diff}<135^\circ$ does the error become very significant? (vSRM sensitivity becomes much less than the SRM sensitivity - this shouldn't affect the optimisation because these configurations will very likely be ruled out)

In [32]:
vary_phiDiff = np.linspace(-180, 180, 361)
rel_err = np.zeros((361,201))
abs_err = np.zeros((361,201))

vSRM_phiComm = 0
vSRM_srcL = 354
default_itmT = 0.01397
default_prmT = 0.03
default_lasPow = 500

SRM_srcL = vSRM_srcL + 4.5
SRM_srmPhi = -90+vSRM_phiComm

for i, phiDiff in enumerate(vary_phiDiff):
    # vSRM sensitivity
    fsig, ASDarr_vSRM, prmT, lasPow = nom.Finesse_sensitivity_into_Bilby(vSRM_phiComm,phiDiff,vSRM_srcL,default_itmT,prmT=default_prmT,lasPow=default_lasPow,optimise_prmT=False,export_peaks=False,squeezing=False)
    SRM_srmT = 0.99985*(1-np.cos(2*phiDiff*np.pi/180)**2)
    kat = finesse.Model()
    # SRM sensitivity (adjust for proper srcL)
    kat.parse(
    f"""
    # NEMO Base Model (simplified from OzHF_ITMloss_4km.kat and converted to Finesse 3)
    ###########################################################################
    ###   Variables
    ###########################################################################
    var Larm 4000
    var Mtm  74.1 # from NEMO paper (94.4 in OzHF_ITMloss_4km.kat)
    var itmT {default_itmT}
    var lmichx 4.5
    var lmichy 4.5 # Changed for symmetry

    ###########################################################################
    ###   Input optics
    ###########################################################################
    l L0 {default_lasPow}

    s l_in L0.p1 prm.p1
    # Power recycling mirror
    m prm T={default_prmT} L=2e-05 phi=90
    s prc prm.p2 bs.p1 L=53

    # Central beamsplitter
    bs bs R=0.4999625 T=0.4999625 alpha=45

    # CHECK Input laser power
    pd P_in L0.p1.o
    # CHECK Laser power incident on BS
    pd P_BS bs.p1.i
    # CHECK PRC Power
    pd P_PRC bs.p1.o

    ###########################################################################
    ###   X arm
    ###########################################################################
    s lx bs.p3 itmxar.p1 L=lmichx

    m itmxar T=1-265.0e-06 L=265.0e-06 phi=180 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
    s ar_thick itmxar.p2 itmx.p1 L=0
    m itmx T=itmT L=20u phi=180
    s LX itmx.p2 etmx.p1 L=Larm

    m etmx T=5u L=20u phi=179.99999

    pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
    pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

    # CHECK X-arm cavity power
    pd P_armX etmx.p1.i

    ###########################################################################
    ###   Y arm
    ###########################################################################
    s ly bs.p2 itmyar.p1 L=lmichy

    m itmyar T=1-265.0e-06 L=265.0e-06 phi=90 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
    s ar_thicky itmyar.p2 itmy.p1 L=0
    m itmy T=itmT L=20u phi=90
    s LY itmy.p2 etmy.p1 L=Larm

    m etmy T=5u L=20u phi=90.00001

    pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
    pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

    # CHECK Y-arm cavity power
    pd P_armY etmy.p1.i

    ###########################################################################
    ###   SRM
    ###########################################################################
    s src bs.p4 srm.p1 L={SRM_srcL}
    m srm R={0.99985-SRM_srmT} T={SRM_srmT} phi={SRM_srmPhi}

    # CHECK SRC power
    pd P_SRC srm.p1.i

    ###########################################################################
    ###   Output & squeezing (from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb)
    ###########################################################################
    dbs OFI 
    link(srm.p2, OFI.p1)
    readout_dc AS OFI.p3.o

    # A squeezed source could be injected into the dark port
    # sq sqz db=7 angle=90
    # link(sqz, OFI.p2)

    # ------------------------------------------------------------------------------
    # Degrees of Freedom
    # ------------------------------------------------------------------------------
    dof STRAIN LX.dofs.h +1  LY.dofs.h -1

    # signal generator
    sgen sig STRAIN

    qnoised NSR_with_RP AS.p1.i nsr=True
    # qshot NSR_without_RP AS.p1.i nsr=True
    # pd1 signal AS.p1.i f=fsig

    fsig(1)
    xaxis(fsig, log, 100, 10k, 200)
    """
    )
    out = kat.run()
    ASDarr_SRM = np.abs(out['NSR_with_RP'])
    abs_err[i,:] = np.abs(ASDarr_SRM-ASDarr_vSRM)
    rel_err[i,:] = np.abs((ASDarr_SRM-ASDarr_vSRM)/ASDarr_vSRM)

    
fig_abs_err = go.Figure(data=go.Contour(z=np.log10(abs_err),x=fsig,y=vary_phiComm))
fig_abs_err.update_layout(title="SRM-vSRM Relations Absolute ASD Error (qnoised)",xaxis_title="Frequency [Hz]",yaxis_title="phiDiff [deg]")
fig_abs_err.update_xaxes(type="log")
fig_abs_err.show()

fig_rel_err = go.Figure(data=go.Contour(z=np.log10(rel_err),x=fsig,y=vary_phiComm))
fig_rel_err.update_layout(title="SRM-vSRM Relations Relative ASD Error (qnoised)",xaxis_title="Frequency [Hz]",yaxis_title="phiDiff [deg]")
fig_rel_err.update_xaxes(type="log")
fig_rel_err.show()

### 3.1.4 Comparing SRM and vSRM configurations (Interactive)  
The similarities between the SRM and vSRM configuration sensitivity curves can be seen directly on this interactive dashboard (SRM configuration tuning and reflectivity calculated from vSRM common- and differential-mode tuning respectively).

In [33]:
import finesse
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import matplotlib.pyplot as plt
from IPython.display import display
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from scipy.signal import find_peaks, peak_widths
from scipy import interpolate


default_srcL = 354
default_phiComm = 0
default_phiDiff = 6.3565
default_itmT = 0.01397
default_prmT = 0.03
default_lasPow = 500

SRM_srcL = default_srcL + 4.5

def plot(phiComm,phiDiff): # This executes when 'Interact' is clicked     
    # Initialise variables (
    # if no peak is found a log error will be thrown for the maximum and half-maximum lines but the sensitivity curves are OK)
    fsig = np.geomspace(100,10e3,201)
    phiComm = float(phiComm)
    phiDiff = float(phiDiff)
    
    fsig, ASDarr_vSRM, prmT, lasPow = nom.Finesse_sensitivity_into_Bilby(default_phiComm,phiDiff,vSRM_srcL,default_itmT,prmT=default_prmT,lasPow=default_lasPow,optimise_prmT=False,export_peaks=False,squeezing=False)
    SRM_srmT = 0.99985*(1-np.cos(2*phiDiff*np.pi/180)**2)
    kat = finesse.Model()
    kat.parse(
    f"""
    # NEMO Base Model (simplified from OzHF_ITMloss_4km.kat and converted to Finesse 3)
    ###########################################################################
    ###   Variables
    ###########################################################################
    var Larm 4000
    var Mtm  74.1 # from NEMO paper (94.4 in OzHF_ITMloss_4km.kat)
    var itmT {default_itmT}
    var lmichx 4.5
    var lmichy 4.5 # Changed for symmetry

    ###########################################################################
    ###   Input optics
    ###########################################################################
    l L0 {default_lasPow}

    s l_in L0.p1 prm.p1
    # Power recycling mirror
    m prm T={default_prmT} L=2e-05 phi=90
    s prc prm.p2 bs.p1 L=53

    # Central beamsplitter
    bs bs R=0.4999625 T=0.4999625 alpha=45

    # CHECK Input laser power
    pd P_in L0.p1.o
    # CHECK Laser power incident on BS
    pd P_BS bs.p1.i
    # CHECK PRC Power
    pd P_PRC bs.p1.o

    ###########################################################################
    ###   X arm
    ###########################################################################
    s lx bs.p3 itmxar.p1 L=lmichx

    m itmxar T=1-265.0e-06 L=265.0e-06 phi=180 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
    s ar_thick itmxar.p2 itmx.p1 L=0
    m itmx T=itmT L=20u phi=180
    s LX itmx.p2 etmx.p1 L=Larm

    m etmx T=5u L=20u phi=179.99999

    pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
    pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

    # CHECK X-arm cavity power
    pd P_armX etmx.p1.i

    ###########################################################################
    ###   Y arm
    ###########################################################################
    s ly bs.p2 itmyar.p1 L=lmichy

    m itmyar T=1-265.0e-06 L=265.0e-06 phi=90 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
    s ar_thicky itmyar.p2 itmy.p1 L=0
    m itmy T=itmT L=20u phi=90
    s LY itmy.p2 etmy.p1 L=Larm

    m etmy T=5u L=20u phi=90.00001

    pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
    pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

    # CHECK Y-arm cavity power
    pd P_armY etmy.p1.i

    ###########################################################################
    ###   SRM
    ###########################################################################
    s src bs.p4 srm.p1 L={SRM_srcL}
    m srm R={0.99985-SRM_srmT} T={SRM_srmT} phi={-90+phiComm}

    # CHECK SRC power
    pd P_SRC srm.p1.i

    ###########################################################################
    ###   Output & squeezing (from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb)
    ###########################################################################
    dbs OFI 
    link(srm.p2, OFI.p1)
    readout_dc AS OFI.p3.o

    # A squeezed source could be injected into the dark port
    # sq sqz db=7 angle=90
    # link(sqz, OFI.p2)

    # ------------------------------------------------------------------------------
    # Degrees of Freedom
    # ------------------------------------------------------------------------------
    dof STRAIN LX.dofs.h +1  LY.dofs.h -1

    # signal generator
    sgen sig STRAIN

    qnoised NSR_with_RP AS.p1.i nsr=True
    # qshot NSR_without_RP AS.p1.i nsr=True
    # pd1 signal AS.p1.i f=fsig

    fsig(1)
    xaxis(fsig, log, 100, 10k, 200)
    """
    )
    out = kat.run()
    ASDarr_SRM = np.abs(out['NSR_with_RP'])
    
    fig_qnoise = go.Figure()
    fig_qnoise.update_xaxes(type="log")
    fig_qnoise.update_yaxes(type="log")
    fig_qnoise.update_layout(title="SRM vs. vSRM Detector Sensitivity",xaxis_title="Frequency [Hz]",yaxis_title="ASD (qnoised) [1/rt Hz]")
    fig_qnoise.add_trace(go.Scatter(x=fsig, y=ASDarr_vSRM,mode='lines+markers',name='vSRM'))
    fig_qnoise.add_trace(go.Scatter(x=fsig, y=ASDarr_SRM,mode='lines+markers',name='SRM'))
    fig_qnoise.show()
    

# Default values here
widgets.interact_manual(plot, phiComm=widgets.Text(value=f"{default_phiComm}"),
                 phiDiff=widgets.Text(value=f"{default_phiDiff}"));

interactive(children=(Text(value='0', continuous_update=False, description='phiComm'), Text(value='6.3565', co…

## 3.2 vSRM Orthogonality
Refer to de Vine, G, Shaddock, DA & McClelland, DE 2002, ‘Variable reflectivity signal mirrors and signal response measurements', Classical and Quantum Gravity, vol. 19, pp. 1561-1568. de Vine's Honours thesis also contains MATLAB code to demonstrate the key result that a SRM in ``signal-recycled Michelson interferometer`` configuration i.e. ``no arm cavities``, can be used to vary the peak frequency and bandwidth "orthogonally". That is, the SRC tuning varies the paek frequency alone, and the SRM reflectivity varies the peak bandwidth alone. In this case, the SRC is formed between ``two`` mirrors (one mirror for the Michelson, and one for the SRM). But, if we refer to Section 2.3.1 and 2.3.2, we see that evidently this orthogonality disappears when arm cavities are added. This is because adding ITMs means a ``three-mirror coupled-cavity`` is formed, where the SRC is formed between the SRM and ITM, and the ITM or ``coupling mirror`` is also part of the arm cavity. I did verify de Vine's results and three-mirror coupled-cavities using Finesse, but those notebooks are very messy and the code in de Vine's work is cleaner.

## 3.3 Peak Properties vs. Tuning 
We co-vary common- and differential-mode tuning and find properties of the first peak (if one exists). We plot against the two-parameters. This process takes a long time so the generating code is commented out (use load and find the associated numpy array in ``./SaveArrays/``).

In [38]:
# import finesse
# import numpy as np
# import plotly.graph_objects as go
# import plotly.express as px
# import pandas as pd
# from scipy.signal import find_peaks, peak_widths
# from scipy import interpolate

# fsig = np.geomspace(100,10e3,201)
vary_phiComm = np.linspace(0.1,10,100)
vary_phiDiff = np.linspace(0.1,10,100)

# peak_sens = np.zeros((100,100))
# peak_f = np.zeros((100,100))
# peak_bw = np.zeros((100,100))

# changed_itmT = True
# store_prmT = 0
# store_lasPow = 0

# # VARY PARAMETER
# for i, phiDiff in enumerate(vary_phiDiff):
#     print(i)
#     for j, phiComm in enumerate(vary_phiComm):
#         if changed_itmT:
#             fsig, ASDarr, prmT, lasPow, peak_dict = nom.Finesse_sensitivity_into_Bilby(phiComm,phiDiff,default_srcL,default_itmT,prmT=0,lasPow=0,optimise_prmT=True,export_peaks=True)
#             store_prmT = prmT
#             store_lasPow = lasPow
#             changed_itmT = False
#             peak_sens[i,j] = peak_dict['peak_sens']
#             peak_f[i,j] = peak_dict['peak_f']
#             peak_bw[i,j] = peak_dict['peak_bw']
#         else: 
#             fsig, ASDarr, prmT, lasPow, peak_dict = nom.Finesse_sensitivity_into_Bilby(phiComm,phiDiff,default_srcL,default_itmT,store_prmT,store_lasPow,optimise_prmT=False,export_peaks=True)
#             peak_sens[i,j] = peak_dict['peak_sens']
#             peak_f[i,j] = peak_dict['peak_f']
#             peak_bw[i,j] = peak_dict['peak_bw']
            
# np.save("SaveArrays/Sec3.3_store_peak_f.npy",peak_f)
# np.save("SaveArrays/Sec3.3_store_peak_bw.npy",peak_bw)
# np.save("SaveArrays/Sec3.3_store_peak_sens.npy",peak_sens)

peak_f = np.load("SaveArrays/Sec3.3_store_peak_f.npy")
peak_bw = np.load("SaveArrays/Sec3.3_store_peak_bw.npy")
peak_sens = np.load("SaveArrays/Sec3.3_store_peak_sens.npy")

fig_peak_f = go.Figure(data=[go.Surface(x=vary_phiComm,y=vary_phiDiff,z=peak_f,opacity=0.5)])
fig_peak_f.update_layout(scene=dict(xaxis=dict(title="phiComm [deg]"),yaxis=dict(title="phiDiff [deg]"),zaxis=dict(title="Peak Frequency [Hz]")))
fig_peak_f.update_layout(title='Peak Frequency vs. vSRM Tuning')
fig_peak_f.show()

fig_peak_bw = go.Figure(data=[go.Surface(x=vary_phiComm,y=vary_phiDiff,z=peak_bw,opacity=0.5)])
fig_peak_bw.update_layout(scene=dict(xaxis=dict(title="phiComm [deg]"),yaxis=dict(title="phiDiff [deg]"),zaxis=dict(title="Peak Bandwidth [Hz]")))
fig_peak_bw.update_layout(title='Peak Bandwidth vs. vSRM Tuning')
fig_peak_bw.show()

fig_peak_sens = go.Figure(data=[go.Surface(x=vary_phiComm,y=vary_phiDiff,z=np.log10(peak_sens),opacity=0.5)])
fig_peak_sens.update_layout(scene=dict(xaxis=dict(title="phiComm [deg]"),yaxis=dict(title="phiDiff [deg]"),zaxis=dict(title="Log Peak Sensitivity [1/rt Hz]")))
fig_peak_sens.update_layout(title='Peak Sensitivity vs. vSRM Tuning')
fig_peak_sens.show()

# Section 4. Optimising Post-Merger Detection Rate 
We define the optimal detector as the detector with the highest detection rate of post-merger signals. A detection is counted as an SNR > 5 which is plausible in the literature (see pp. 10 of [Dietrich et al. 2020] for further details). SNR is calculated using bilby by injecting NR (strain) waveforms of BNS mergers (cropped to the post-merger phase) into each possible detector. The detector response depends on the injection parameters e.g. distance from merger event SNR $\propto 1/d$. All detection rates below are for single detectors i.e. not in a network! 
**Instructions**: Use the notebook ``ResultsSupplementary.ipynb`` to setup the CoRE database and ensure Mallika's code is in the parent directory. Also ensure that bilby is version 1.1.3! (newer versions will require NS masses in injection parameters)

**TODO**: 

1. Explore alternative figure of merit(s) for possible detector designs. For example, a simple extension would be to maximise the detection rate *in a network* with other interferometric detectors.  

2. A lot of the functions in the ``BNS_Optimisation_Module_Randomised`` module could be rewritten using the module watpy recommended by the authors of the CoRe database to future-proof the code.  

3. Correct waveform scaling to make use of the full waveform set available.

4. Calculate detection rate in a network!

In [39]:
import finesse
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import nemo_optimisation_modules as nom
import plotly.io as pio